In [1]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [2]:


llm = Ollama(model="qwen2:7b-instruct-q6_K", request_timeout=60.0)
embed_model = OllamaEmbedding(
    model_name="all-minilm:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [4]:
import os


os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "localhost:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [5]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

# graph_store = NebulaGraphStore(
#     space_name=space_name,
#     edge_types=edge_types,
#     rel_prop_names=rel_prop_names,
#     tags=tags,
# )
# storage_context = StorageContext.from_defaults(graph_store=graph_store)

from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

In [6]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config


# Initialize connection pool
config = Config()
config.max_connection_pool_size = 10
connection_pool = ConnectionPool()
assert connection_pool.init([("localhost", 9669)], config)

# Create a session
with connection_pool.session_context("root", "nebula") as session:
    # Create a space
    create_space_query = f"""
    CREATE SPACE IF NOT EXISTS {space_name} (partition_num=15, replica_factor=1, vid_type=FIXED_STRING(3000));
    """
    session.execute(create_space_query)

    # Use the newly created space
    use_space_query = "USE test_space;"
    session.execute(use_space_query)

    print(f"Space '{space_name}' created and selected successfully.")

# Close the connection pool
connection_pool.close()

Space 'llamaindex' created and selected successfully.


In [14]:
from llama_index.core import KnowledgeGraphIndex
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    llm=llm,
    verbose=True,
)

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text="Guardians of the Galaxy Vol. 3 (marketed as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios and di

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text="The Guardians reach Counter-Earth and are guided to the Arête Laboratories complex. Drax and Mantis remain with Gamora and Rocket, while Peter Quill, Groot, and Nebula travel to Arête. Nebula is forced to wait ou

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='Dave Bautista as Drax the Destroyer:A member of the Guardians and a highly skilled warrior whose family was slaughtered by Ronan the Accuser, under the instructions of Thanos. Bautista stated that Vol. 3 would be

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='Gunn likened the High Evolutionary to "a space version" of Doctor Moreau from Island of Lost Souls (1932), a film Gunn is a big fan of, calling him "a detestable character". Gunn described the High Evolutionary a

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='He also reiterated that he had not decided whether he would be directing the film, and that he was going to figure out his involvement and his next project "over the next couple of weeks". Part of Gunn\'s relucta

...

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='In August, Gunn turned in a new draft of the film\'s script and began writing a spin-off television series from The Suicide Squad titled Peacemaker (2022). A month later, he was planning to begin work on Vol. 3 i

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='Gunn also reiterated his comments on not resurrecting Yondu in the film.\n\n\n=== Filming ===\nPrincipal photography began on November 8, 2021, at Trilith Studios in Atlanta, Georgia, under the working title Hot 

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='== Music ==\n\nIn April 2017, Gunn felt the music for the film would be different from what was used for the first two films\' soundtracks, Awesome Mix Vol. 1 and Vol. 2. The next month, he added that he was "pan

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text="=== Home media ===\nGuardians of the Galaxy Vol. 3 was released by Walt Disney Studios Home Entertainment on digital download on July 7, 2023; on Ultra HD Blu-ray, Blu-ray, and DVD on August 1; and on Disney+ on 

self.kg_triple_extract_template=PromptTemplate(metadata={'prompt_type': <PromptType.KNOWLEDGE_TRIPLET_EXTRACT: 'knowledge_triplet_extract'>}, template_vars=['max_knowledge_triplets', 'text'], kwargs={'max_knowledge_triplets': 10}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Some text is provided below. Given the text, extract up to {max_knowledge_triplets} knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n---------------------\nExample:Text: Alice is Bob's mother.Triplets:\n(Alice, is mother of, Bob)\nText: Philz is a coffee shop founded in Berkeley in 1982.\nTriplets:\n(Philz, is, coffee shop)\n(Philz, founded in, Berkeley)\n(Philz, founded in, 1982)\n---------------------\nText: {text}\nTriplets:\n")
text='A negative review from NPR criticized the approach of cruelty to animals in the film while The New York Times called it a "dour, visually off-putting two-and-a-half-hour A.S.P.C.A. nightmare of a film (that) may 

# Test query
## Initial session

In [66]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config
from nebula3.gclient.net import Session

# Configuration
nebula_host = "127.0.0.1"  # Replace with your Nebula Graph host
nebula_port = 9669  # Replace with your Nebula Graph port
username = "root"  # Replace with your Nebula Graph username
password = "nebula"  # Replace with your Nebula Graph password
space = "llamaindex"  # Replace with your Nebula Graph space name

# Create a configuration object
config = Config()
config.max_connection_pool_size = 10

# Initialize the connection pool
connection_pool = ConnectionPool()
assert connection_pool.init([(nebula_host, nebula_port)], config), "Failed to initialize the connection pool"

# Create a session
session = connection_pool.get_session(username, password)
assert (session := connection_pool.get_session(username, password)), "Failed to create a session"

## Query

In [104]:
import pandas as pd


def run_query(space: str, query: str):

    # Use the space
    session.execute(f"USE {space}")


    # Execute the query
    result = session.execute(query)

    # Check if the query was successful
    if not result.is_succeeded():
        raise ValueError("Query failed: {}".format(result.error_msg()))

    # Process the results
    return result.as_data_frame(primitive=True)

    

def post_process_raw_data(raw_df: pd.DataFrame) -> pd.DataFrame:
    df = pd.DataFrame()

    df["entity_1"] = raw_df["p"].apply(lambda p: p["tags"]["entity"]["name"]).astype(str)
    df["relationship"] = raw_df["r"].apply(lambda r: r["props"]["relationship"]).astype(str)
    df["entity_2"] = raw_df["q"].apply(lambda q: q["tags"]["entity"]["name"]).astype(str)
    return df


def get_entity_relation(space: str, query: str) -> pd.DataFrame:
    raw_df = run_query(space, query)
    relation_df = post_process_raw_data(raw_df)
    return relation_df

def get_query(entity_name: str) -> str:
    """Get ngql query
    # TODO: Add Sanitizing
    
    """
    query = f"""MATCH (p:entity)-[r:relationship]->(q:entity)
WHERE (
    LOWER(p.entity.name) CONTAINS LOWER("{entity_name}") OR
    LOWER(q.entity.name) CONTAINS LOWER("{entity_name}")
)
RETURN p,r,q LIMIT 10;"""
    return query

query = get_query("The guardians")
relation_df = get_entity_relation(space, query)
relation_df

,entity_1,relationship,entity_2
0,"""Callie brand""","""Appears in""","""The guardians of the galaxy holiday special"""
1,"""Warlock""","""Described by""","""More traditional superhero compared to the gu..."
2,"""Gunn""","""Thought""","""Warlock's interactions with the guardians pro..."
3,"""The interior of the guardians' new ship""","""The bowie""","""Was a four-story set"""
4,"""The guardians""","""Travel to""","""Orgocorp's headquarters"""
5,"""The guardians""","""Destination for""","""Finding theel"""
6,"""The guardians""","""Reach""","""Counter-earth"""
7,"""The guardians of the galaxy holiday special""","""Provides relief for actors compared to""","""Vol. 3"""
8,"""The guardians of the galaxy holiday special""","""Filmed for""","""February to late april 2022"""
9,"""Ayesha and adam""","""Pursued by""","""The guardians"""


In [110]:
relation_df["context"] = (
    relation_df["entity_1"]
    + " "
    + relation_df["relationship"]
    + " "
    + relation_df["entity_2"]
)
print("- " + "\n- ".join(relation_df["context"]))

- "Callie brand" "Appears in" "The guardians of the galaxy holiday special"
- "Warlock" "Described by" "More traditional superhero compared to the guardians"
- "Gunn" "Thought" "Warlock's interactions with the guardians provided an interesting juxtaposition to their journey"
- "The interior of the guardians' new ship" "The bowie" "Was a four-story set"
- "The guardians" "Travel to" "Orgocorp's headquarters"
- "The guardians" "Destination for" "Finding theel"
- "The guardians" "Reach" "Counter-earth"
- "The guardians of the galaxy holiday special" "Provides relief for actors compared to" "Vol. 3"
- "The guardians of the galaxy holiday special" "Filmed for" "February to late april 2022"
- "Ayesha and adam" "Pursued by" "The guardians"


In [123]:
import ollama

respones = ollama.chat(
    model="qwen2:7b-instruct-q6_K",
    messages=[
        {"role": "system", "content": "Extract the main entity from the user's question. You must return the query entity only and not include prologue, prefix or suffix"},
        {"role": "system", "content": "Tell me about to the guardians"}
    ]
)
keyword = respones["message"]["content"]
query = get_query(keyword)
relation_df = get_entity_relation(space, query)
relation_df

,entity_1,relationship,entity_2
0,"""Music""","""Genre is different in""","""Guardians of the galaxy vol. 3"""
1,"""Phase five of the mcu""","""Part of""","""Guardians of the galaxy vol. 3's release"""
2,"""The three guardians films""","""Work together as a whole""","""Telling one story"""
3,"""Gunn confirmed in september 2019""","""Intended for vol. 3 to be""","""His last guardians film"""
4,"""Mantis""","""Is a member of""","""Guardians"""
5,"""Callie brand""","""Appears in""","""The guardians of the galaxy holiday special"""
6,"""Mc""","""Contains film in series""","""Guardians of the galaxy vol. 3"""
7,"""Warlock""","""Described by""","""More traditional superhero compared to the gu..."
8,"""Marvel studios""","""Produces""","""Guardians of the galaxy vol. 3"""
9,"""Ayesha and adam""","""Pursued by""","""The guardians"""


guardians


In [16]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))


ValidationError: 1 validation error for LLMPredictStartEvent
template
  none is not an allowed value (type=type_error.none.not_allowed)

In [84]:
print(documents[0].text)

Guardians of the Galaxy Vol. 3 (marketed as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the other Guardians race to save the life of Rocket (Cooper) from his creator, the High Evolutionary (Iwuji), who is an alien scientist trying to perfect the universe.
Gunn had initial ideas for a third and final Guardians of the Galaxy film by November 2014, and announced his return to write and d